# How do we prepare data for a bike share analytics platform?

## Exploring the data

Dealing with unclean data is not necessarily glamorous, yet it is always essential. A common example would be a repetitive typo; e.g. 1% of the rows having "New York" spelled as "New Yok". We can use a combination of `.groupby()` and `.count()` in order to get a summary of the different values in a particular column, and notice that there are some common misspelled proper nouns.

It can also be a good habit to shuffle the data and sift through a few hundred/thousand rows manually to get a good "feel" for the data. While this may seem excessive, you may catch an insight that would dramatically accelerate your subsequent work. Learn to love manually digging into the data if you want to be a data professional.

Let's read the data using the `.read_csv()` method. Note that we are passing a parameter `nrows=10000` so that we only read in the first 10,000 rows in the file (we are working with a subset of the data on our local machine as the entire dataset is quite large - more than 3 million rows):

In [ ]:
import numpy as np
import pandas as pd
import math
import base64

In [ ]:
# Read the data file and take a look at the data
df = pd.read_csv('data/trips.csv', nrows=10000)
df.head()

As discussed before, the first step we ought to take with a new dataset is to familiarize ourselves with it. Let's read through the columns present in the dataset, find out how the data is spread out across columns, etc. This will also give us a sense of the obvious cleaning steps to be performed on each column present in the dataset.

In [ ]:
# Take a look at the columns
df.columns

The list of available features is as follows:

1. **Rental Id:** A user can purchase the right to pick any bike in the city within 24 hours for £2. This column contains the ID of the rental (which may correspond to more than one bike). Notice that this is *not* the ID of the journey. Data type: `int64`.
1. **Duration:** The duration of the journey in seconds. Data type: `int64`.
2. **Bike Id:** The ID of the bike. Data type: `int64`.
3. **End Date:** End time of the journey. Data type: `object`.
4. **EndStation Id:** ID of the station at which this journey ended. Data type: `int64`.
5. **Start Date:** Start time of the journey. Data type: `object`.
6. **StartStation Id:** ID of the station at which this journey started. Data type: `int64`.
7. **tag:** A tag that one of the members of your team assigned to each journey to make it easier to group them for further analysis. This column was not part of the original dataset. Data type: `object`.
8. **userCategory:** Can be either `A` (occasional user) or `B` (frequent user). Data type: `object`.

## Format

![Format icon](data/images/format.jpg)

As mentioned during lecture, we will be handing our dataset as a `.csv` file encoded using `UTF-8`. We will make sure that these two requirements are met by exporting our file with the `df.to_csv("clean.csv", encoding="utf-8")` command.

## Relevance

![Relevance icon](data/images/relevance.jpg)

We said that we needed to:

* Replace each missing `Bike Id` with the "Not available" string
* Interpolate the missing values in the `userCategory` column
* Replace each missing `EndStation Id` with the "Not available" string

### Example 1

Replace the missing values in `bike_id` and `end_station_id` with the "Not Available" string.

**Answer.** There are a number of ways to do this, but we will do so with the [**`.fillna()`**](https://pandas.pydata.org/docs/reference/api/pandas.Series.fillna.html) Series function. This function takes a single argument and replaces all `NaN` values in the Series with the value of that argument:

In [ ]:
df["Bike Id"] = df["Bike Id"].fillna('Not available')
df["EndStation Id"] = df["EndStation Id"].fillna('Not available')

### Interpolating missing values

To do the interpolation for `userCategory`, we can use the [**`.interpolate()`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.interpolate.html) method in `pandas`. The `interpolate()` method uses linear interpolation, which is a mathematical method for filling in unknown points based on building a linear regression model on the non-missing points (you will learn more about linear regression in future cases). We can then use this model to estimate the values of the missing points. This is *very* different from substituting null values with random or meaningless values, as it *preserves aspects of the distribution of the data, which can be very important for certain analyses*.

But `userCategory` is a string - how can we apply a mathematical model to a string? Luckily, `userCategory` can only take on two values, so we can convert it to a `category` type with `.astype()` and then run the interpolation:

In [ ]:
# Convert to a category type
df['userCategory'] = df['userCategory'].astype('category')

After converting `userCategory` to be a category column, let's see what the data looks like:

In [ ]:
# The categories
df['userCategory'].cat.categories

In [ ]:
# Let's see the codes assigned to the categories
df['userCategory'].cat.codes.unique()

You can see there are 2 codes here: 0 and 1, which correspond to the two values `['A']` and `['B']`. (The number -1 represents the NaN values.) In order to interpolate the values, we need to call the `.interpolate()` method after having replaced the -1s with actual null values (`np.nan`s):

In [ ]:
# The below code replaces the value -1 with NaN
user_cat_codes = df['userCategory'].cat.codes.replace(-1, np.nan)

# We now call the interpolate function that actually fills the NaN values with either a 0 or 1
user_cat_codes = user_cat_codes.interpolate()
user_cat_codes

Now we convert the codes to the `category` data type:

In [ ]:
user_cat_codes = user_cat_codes.astype(int).astype('category')

Finally, we replace back the 0s and 1s with the actual category names. For this, we can use the **`.cat.rename_categories()`** method:

In [ ]:
user_cat_codes = user_cat_codes.cat.rename_categories(df['userCategory'].cat.categories)
df['userCategory'] = user_cat_codes
df['userCategory']

## Consistency

![Consistency icon](data/images/consistency.jpg)

### Data type consistency

From our previous examination of the dataset, we decided that these changes had to be made:

| Column | Current data type | Convert to |
| --- | --- | --- |
| `Rental Id` | `int64` | `category` |
| `Bike Id` | `int64` | `category` |
| `End Date` | `object` | `datetime` |
| `EndStation Id` | `int64` | `category` |
| `Start Date` | `object` | `datetime` |
| `StartStation Id` | `int64` | `category` |
| `tag` | `object` | `category` |
| `userCategory` | `object` | `category` |

### Exercise 1

To change the data type of a Series in `pandas`, we use the `.astype()` method that you learned in previous cases. Make the changes for all the columns except for `Start Date` and `End Date` (we will explain how to do those shortly). You also do not need to convert `userCategory` since we already did that when we interpolated its missing values.

**Answer.**

-------

### Converting dates

This is a typical date in the dataset: `12/08/2020 16:28`. It follows the `DD/MM/YYYY` date format, and the 24 hour time format. The date and the time are separated by a space. To convert this kind of data into a `datetime` column, we use the **`pd.to_datetime()`** function like this (this function automatically detects the format, so we don't have to worry about that):

In [ ]:
df["Start Date"] = pd.to_datetime(df["Start Date"])
df["End Date"] = pd.to_datetime(df["End Date"])

### Exercise 2

In terms of unit consistency, we need to create a new column, `duration_min`, that is the duration of the journeys in minutes instead of seconds. Create the column in `df`.

**Answer.**

-------

Let's now remove the `Duration` column:

In [ ]:
df = df.drop(columns=["Duration"])

### Categorical consistency

We noticed that in `tag` we are supposed to have only three categories, `low`, `medium`, and `high`, but according to our report, we have five. A closer look at our DataFrame reveals that we have these labels: "priority low", "Priority low", "priority_high", "priority_medium", and "priority Medium". We need to standardize these. We can make a replacement dictionary and rename our categories with this code:

In [ ]:
rename_dict = {"Priority low":"low",
               "priority_high":"high",
               "priority Medium":"medium",
               "priority_medium":"medium",
               "priority low":"low"
              }
df["tag"] = df["tag"].replace(rename_dict).astype("category")
df["tag"]

### Exercise 3

Do the appropriate replacements for the `userCategory` column (it has some weird formatting - `['A']` and `['B']` instead of `A` and `B`).

**Answer.**

-------

### Referential integrity

We now need to remove the duplicates in the dataset. The method of choice is [**`.drop_duplicates()`**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html). When you use this method on a Series or DataFrame, it removes all the duplicate rows (keeping only the first occurrence) and then outputs the Series or DataFrame:

In [ ]:
df = df.drop_duplicates()

The default in `.drop_duplicates()` when used on a DataFrame is to treat as duplicates only the rows that are equal in all of their fields, but you can also ask `pandas` to check for duplicity only on a subset of the fields, using the `subset` argument.

We also need to create trip IDs. When generating unique identifiers for datasets like this, we should make sure the generation process is **idempotent** (i.e. the same ID should be generated for each trip no matter how many times you run the script). This is required because there may be chances that the same trip is inputted into this tool multiple times. For example, suppose the customer first uploads the dataset for the first week of the month (for testing purposes, or based on data availability, etc.), and later uploads the dataset for the entire month. Now, if the same trip is assigned different IDs for each upload, then it might result in the analytics platform interpreting this as two different trips and this will skew the analysis.

### Exercise 4

Describe how you would generate a unique ID per trip while guaranteeing idempotence, then write code to do this.

**Answer.**

-------

### Column name consistency

### Example 2

Rename the columns so that they are in all-lowercase and separate words with underscores.

**Answer.** We can use the **`.rename()`** method to do this efficiently. Similar to `replace()`, this method can also take a dictionary as its argument to compress the amount of necessary code-writing:

In [ ]:
columns_dict = {"Rental Id":"rental_id",
                "Duration":"duration",
                "Bike Id":"bike_id",
                "End Date":"end_date",
                "EndStation Id":"end_station_id",
                "Start Date":"start_date",
                "StartStation Id":"start_station_id",
                "userCategory":"user_category"
               }
df = df.rename(columns=columns_dict)
df.columns

## Data augmentation

![Data augmentation](data/images/data_augmentation.jpg)

Our priorities are:

1. Merging `trips.csv` with `stations.json`
2. Creating the following features:
    1. Day of the week for both `start_date` and `end_date`
    2. Hour for both `start_date` and `end_date`
    3. Total cost of the rental
    4. Location (the general area where the station is located). In the `stations.json` file, these are the strings after the comma in `Station name`.

### Merging the datasets

To merge the two datasets, we first need to load `stations.json` into `pandas`. For this, we will use the [**`pd.read_json()`**](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html) function:

In [ ]:
stations = pd.read_json("data/stations.json", orient="columns")
stations.head()

**JSON files** are very similar to Python dictionaries. They are basically text files whose content is a dictionary that can be read by any major programming language.

JSON files are sometimes used instead of CSV files to share data because in some cases they take up less memory. With `pd.read_json()`, you can read files like this and turn them into DataFrames with a single line of code. Depending on how the JSON file is built, you can specify the orientation with the `orient` argument. Above, we used `orient="columns"` because the `stations.json` file's structure follows this pattern:

~~~json
{
   "col 1":{
      "row 1":"This is column 1, row 1",
      "row 2":"This is column 1, row 2"
   },
   "col 2":{
      "row 1":"This is column 2, row 1",
      "row 2":"This is column 2, row 2"
   }
}
~~~

But let's say you have a file like this:

~~~json
{
   "row 1":{
      "col 1":"This is row 1, column 1",
      "col 2":"This is row 1, column 2"
   },
   "row 2":{
      "col 1":"This is row 2, column 1",
      "col 2":"This is row 2, column 2"
   }
}
~~~

The correct way to read it would be using `orient="index"` (since the default integer indexes in a DataFrame refer to rows, this makes logical sense).

In [ ]:
j = """{
   "row 1":{
      "col 1":"This is row 1, column 1",
      "col 2":"This is row 1, column 2"
   },
   "row 2":{
      "col 1":"This is row 2, column 1",
      "col 2":"This is row 2, column 2"
   }
}"""

pd.read_json(j, orient="index") # Try changing "index" for "columns" to see what happens!

We can create the `location` column here before doing the merge. For that, we use the [**`.str.split()`**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html) method. This method takes all the strings of a string Series and splits them using instances of a particular character within those strings. One common use case is when you have a column with strings that look like this:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>my_series</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Houston,Texas,United States</td>    </tr>    <tr>      <th>1</th>      <td>Toronto,Ontario,Canada</td>    </tr>    <tr>      <th>2</th>      <td>Tucson,Arizona,United States</td>    </tr>  </tbody></table>


You can split these values using the comma as a separator with the code `my_series.str.split(pat=",")` to get this result:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>my_series_split</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>[Houston, Texas, United States]</td>    </tr>    <tr>      <th>1</th>      <td>[Toronto, Ontario, Canada]</td>    </tr>    <tr>      <th>2</th>      <td>[Tucson, Arizona, United States]</td>    </tr>  </tbody></table>

Notice how each string in this Series was turned into a list, where each element of this list was derived from the characters between consecutive instances of the "," character (or between the start of the string and the first "," for the first element, and the last "," and the end of the string for the last element).

If you want the split values to be represented as columns of a DataFrame instead of lists inside a column, we can add the `expand=True` argument like this: `my_series.str.split(pat=",", expand=True)`. This is the result:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>0</th>      <th>1</th>      <th>2</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Houston</td>      <td>Texas</td>      <td>United States</td>    </tr>    <tr>      <th>1</th>      <td>Toronto</td>      <td>Ontario</td>      <td>Canada</td>    </tr>    <tr>      <th>2</th>      <td>Tucson</td>      <td>Arizona</td>      <td>United States</td>    </tr>  </tbody></table>

Let's now apply this to our read-in JSON file:

In [ ]:
split_columns = stations["Station Name"].str.split(pat=",", expand=True)
split_columns.head()

Whoops! Somehow a third column got appended and it seems to be `None` for all of the rows. How can this be? Well, if this is happening, it's likely that there is *some* station name which has TWO commas in it, and so it is getting split properly into three columns. But most other station names only have ONE comma in them, so splitting those into three columns would necessarily mean the last column would be empty. Let's see if we're right by running the following:

In [ ]:
split_columns[2].value_counts()

It seems there are two stations with two commas in them. In order to deal with this, we use the `n` argument, which tells `pandas` the maximum number of splits `str.split()` should perform. For instance, you could use `my_string.str.split(pat=",", n=1, expand=True)` to split only at the first comma and get this result. Let's apply this to our read-in JSON file:

In [ ]:
split_columns = stations["Station Name"].str.split(pat=",", n=1, expand=True)
split_columns.head()

There we go! Now we simply add these columns to `stations` and do the corresponding adjustments:

In [ ]:
stations = pd.concat([stations, split_columns], axis=1)
rename_dict = {
    0:"station_name",
    1:"location",
    "Latitude":"latitude",
    "Longitude":"longitude",
    "Station ID":"station_id"
}
stations = stations.rename(columns=rename_dict)
stations = stations.drop(columns=["Station Name"])
stations["station_id"] = stations["station_id"].astype("category") # We need this to be a category feature
stations.head()

We also use **`str.strip()`** to get rid of leading and trailing spaces:

In [ ]:
stations["station_name"] = stations["station_name"].str.strip()
stations["location"] = stations["location"].str.strip()

The [`str.strip()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.strip.html) method takes all the strings in a Series and removes any leading or trailing whitespaces in them. After being processed with this method, a string like "   hello world! " will, for instance, be stripped to become "hello world!".

You can also specify a custom character to remove with the `to_strip` argument (e.g. to remove `_`s you would use `my_series.str.strip(to_strip="_")`, and "\_hello world!\_" would become "hello world!"), rather than removing whitespaces.

To merge the datasets, we use `pd.merge()`. For `start_station_id`:

In [ ]:
df = pd.merge(df, stations, left_on="start_station_id", right_on="station_id", how="left")

rename_dict = {
    "latitude":"start_latitude",
    "longitude":"start_longitude",
    "station_name":"start_station_name",
    "location":"start_location"
}
df = df.rename(columns=rename_dict)
# Remove the extra column that was added
df = df.drop(columns=["station_id"])
# Fill any remaining nulls with "Not Available" - only for string columns
obj_cols = df.columns[df.dtypes=="object"]
df[obj_cols] = df[obj_cols].fillna('Not Available')
df.head()

### Exercise 5

Copy the code above and paste it in a new cell. Modify it to do the merge for the end stations this time.

**Answer.**

-------

### Feature engineering

Finally, let's create the `start_weekday`, `start_hour`, and `rental_cost` features. We start with the first two:

In [ ]:
# We've generated the merged df DataFrame for you, but you will have to figure out the code we used (in Exercise 5)!
df = pd.read_csv("data/df_exercise.csv", parse_dates=["start_date"])

df['start_hour'] = df['start_date'].dt.hour # dt.hour gets you the hour
df['start_weekday'] = df['start_date'].dt.weekday # dt.weekday gets you the weekday (as an integer)

Here we used the [**`dt.hour`**](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.hour.html) and [**`dt.weekday`**](https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.weekday.html) attributes. These only work with columns that are of type `datetime`. These basically acess the hour and the weekday of the `datetime`. (Observe that you have to add `dt` before you call these attributes.) Other attributes include `dt.tz` (timezone), `dt.year`, `dt.month`, `dt.day`, `dt.minute`, and `dt.second`, [amongst others](https://pandas.pydata.org/docs/reference/series.html#datetimelike-properties).

### Exercise 6 (optional)

Create the `rental_cost` DataFrame. Remember that you have to pay £2 upfront for your rental, and then you can have as many rides as you want without paying more, so long as you don't exceed 30 minutes in each ride. If you exceed this threshold, you have to pay £2 for each 30-minute block (a fraction of a block counts as a full block).

The DataFrame should look like this:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>rental_id</th>      <th>rental_cost</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>101368080</td>      <td>2</td>    </tr>    <tr>      <th>1</th>      <td>101368081</td>      <td>2</td>    </tr>    <tr>      <th>2</th>      <td>101368082</td>      <td>2</td>    </tr>    <tr>      <th>3</th>      <td>101368083</td>      <td>2</td>    </tr>    <tr>      <th>4</th>      <td>101368084</td>      <td>2</td>    </tr>    <tr>      <th>5</th>      <td>101368086</td>      <td>2</td>    </tr>    <tr>      <th>6</th>      <td>101368172</td>      <td>2</td>    </tr>    <tr>      <th>7</th>      <td>101368173</td>      <td>2</td>    </tr>    <tr>      <th>8</th>      <td>101368174</td>      <td>2</td>    </tr>    <tr>      <th>9</th>      <td>101368176</td>      <td>2</td>    </tr>    <tr>      <th>...</th>      <td>...</td>      <td>...</td>    </tr>  </tbody></table>

**Hint:** To round a float number up, you can use the [`.ceil()`](https://www.w3schools.com/python/ref_math_ceil.asp) function from the `math` module.

**Answer.**

-------

### Joining the `rental_cost` data (optional)

Now, we can merge the two DataFrames and make `trip_id` the index. We can do this using the `.set_index()` DataFrame method:

In [ ]:
# We've generated the rental_cost DataFrame for you, but you will have to figure out the code we used (in Exercise 6)!
rental_cost = pd.read_csv("data/rental_cost.csv")

df = pd.merge(df, rental_cost, left_on="rental_id", right_on="rental_id", how="left")
df = df.set_index("trip_id")
df.head()

## Exporting the dataset

Our dataset is now clean and ready to be exported and delivered to our client. We save it to our working directory using the [**`.to_csv()`**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) method:

In [ ]:
df.to_csv("clean.csv", encoding="utf-8")

This method can take many arguments (check the [docs](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) for details). Here we only used the desired filename (`"clean.csv"`) and `encoding="utf-8"`, to make sure the result uses UTF-8 (this is the default value, so we could have also left this argument unspecified). If we wanted our file to be saved to a subfolder (say, `results/`), we could run `df.to_csv("results/clean.csv", encoding="utf-8")` instead.